In [24]:
# =========================================================================
# PHASE 3: TIME SERIES MODELING & CAUSALITY (FULL ADJUSTMENT)
# =========================================================================

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.statespace.sarimax import SARIMAX
from causalimpact import CausalImpact
import os
import matplotlib.pyplot as plt

# --- Define File Path ---
# Assuming the final imputed CSV from the previous step is the input
FINAL_IMPUTED_FILE = '../data/merged_timeseries_final_imputed.csv' 

# Load the merged and imputed data
# NOTE: If your imputation step saved the file as 'merged_timeseries.csv' 
# in the data folder, update the file name here.
df = pd.read_csv(FINAL_IMPUTED_FILE, index_col='date', parse_dates=True).dropna()

df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize(None) 
df.index.name = 'date'

# --- Define Variables for Modeling ---
DEPENDENT_VAR = 'nft_sales_volume_eth'
PRIMARY_EXOG = ['avg_gas_price_gwei'] # The cause variable (X)

# All Confounders (C): ETH Price, Sentiment Score, New Addresses
CONFOUNDERS = ['eth_price_usd', 'fear_greed_score', 'new_addresses']
ALL_EXOG_VARS = PRIMARY_EXOG + CONFOUNDERS

In [25]:
# --- Corrected Chunk 2: Stationarity and Controlled Granger Causality ---

# Import for the final statistical test (scipy is usually standard)
from scipy.stats import f # Used for the F-Test calculation

# --- (a) Stationarity Check (ADF Test) ---
print("--- ADF Test for Stationarity (Determining Integration Order 'd') ---")

def check_stationarity(series, name):
    # Use 1st difference (series.diff().dropna())
    result = adfuller(series.diff().dropna()) 
    print(f'{name} p-value: {result[1]:.4f}')
    
check_stationarity(df[DEPENDENT_VAR], 'NFT Volume')
check_stationarity(df['avg_gas_price_gwei'], 'Gas Price')


# --- (b) Controlled Granger Causality Test (Manual OLS F-Test using RSS) ---
# Goal: Test if Gas Price (X) predicts NFT Volume (Y), controlling for Confounders (C).
print("\n--- Granger Causality Test: Gas Price -> NFT Volume (MANUALLY Adjusted) ---")

# 1. Prepare Data
# The analysis requires all variables to be stationary. Use the 1st difference (d=1).
data_diff = df[[DEPENDENT_VAR] + PRIMARY_EXOG + CONFOUNDERS].diff().dropna()
LAG = 7 # We test for causality at 7 days

y_var = data_diff[DEPENDENT_VAR]
X_vars = data_diff[PRIMARY_EXOG + CONFOUNDERS]

# 2. Build the Restricted Model (H0: X does NOT cause Y)
restricted_X = []
for i in range(1, LAG + 1):
    restricted_X.append(y_var.shift(i).fillna(0)) # Lagged Y
    for col in CONFOUNDERS:
        restricted_X.append(X_vars[col].shift(i).fillna(0)) # Lagged Confounders (C)

restricted_X = pd.concat(restricted_X, axis=1).iloc[LAG:]
restricted_X = sm.add_constant(restricted_X, prepend=False)
restricted_y = y_var[LAG:]

# Fit the Restricted Model
restricted_model = sm.OLS(restricted_y, restricted_X).fit()


# 3. Build the Unrestricted Model (H1: X DOES cause Y)
unrestricted_X = []
for i in range(1, LAG + 1):
    unrestricted_X.append(y_var.shift(i).fillna(0)) # Lagged Y
    for col in CONFOUNDERS:
        unrestricted_X.append(X_vars[col].shift(i).fillna(0)) # Lagged C
    for col in PRIMARY_EXOG:
         unrestricted_X.append(X_vars[col].shift(i).fillna(0)) # Lagged X (Gas Price)

unrestricted_X = pd.concat(unrestricted_X, axis=1).iloc[LAG:]
unrestricted_X = sm.add_constant(unrestricted_X, prepend=False)

# Fit the Unrestricted Model
unrestricted_model = sm.OLS(restricted_y, unrestricted_X).fit()


# 4. Perform the F-Test Manually using Residual Sum of Squares (RSS)
RSS_R = restricted_model.ssr # RSS of the Restricted Model
RSS_U = unrestricted_model.ssr # RSS of the Unrestricted Model

q = len(PRIMARY_EXOG) * LAG # Number of restrictions (lags * variables tested)
T = len(restricted_y) # Number of observations
k = unrestricted_model.df_model + 1 # Number of parameters in the unrestricted model

# F-statistic formula: ((RSS_R - RSS_U) / q) / (RSS_U / (T - k))
F_statistic = ((RSS_R - RSS_U) / q) / (RSS_U / (T - k))

# Calculate the P-value from the F-distribution
p_value_adjusted = f.sf(F_statistic, q, T - k) # f.sf is the survival function (1 - CDF)

print(f"Adjusted Granger Test Results (Lag {LAG}):")
print(f"F-statistic: {F_statistic:.4f}")
print(f"P-value (Adjusted): {p_value_adjusted:.4f}")

if p_value_adjusted < 0.05:
    print("\nCONCLUSION: P < 0.05. We reject H0. Gas Price **DOES** Granger-cause NFT Volume, even when controlling for Confounders.")
else:
    print("\nCONCLUSION: P >= 0.05. We fail to reject H0. No Adjusted Granger Causality detected.")

# --- End of Chunk 2 ---

--- ADF Test for Stationarity (Determining Integration Order 'd') ---
NFT Volume p-value: 0.0000
Gas Price p-value: 0.0000

--- Granger Causality Test: Gas Price -> NFT Volume (MANUALLY Adjusted) ---
Adjusted Granger Test Results (Lag 7):
F-statistic: 1.6887
P-value (Adjusted): 0.1167

CONCLUSION: P >= 0.05. We fail to reject H0. No Adjusted Granger Causality detected.


In [26]:
# --- (c) ARIMAX Model Fitting (Full Adjustment) ---
# Model isolates the coefficient of 'avg_gas_price_gwei' (Primary Exogenous)
print("\n--- ARIMAX Model: NFT Volume (Y) explained by Gas Price (X) [Full Adjustment] ---")

endog = df[DEPENDENT_VAR]
exog = df[ALL_EXOG_VARS] # Includes Gas Price AND Confounders

# Fit the SARIMAX model: (p, d, q) = (1, 1, 1) is a common starting point
# d=1 is used because we anticipate non-stationarity (as confirmed by ADF likely).
model = SARIMAX(endog, exog=exog, order=(1,1,1), enforce_stationarity=False, enforce_invertibility=False)
results = model.fit(disp=False)
print(results.summary())

# Key Interpretation: Look at the coefficient for the primary variable
print(f"\nADJUSTED Gas Price Coefficient (Isolated Effect): {results.params['avg_gas_price_gwei']:.4f}")
print(f"Fear & Greed Score Coefficient (Confounder Control): {results.params['fear_greed_score']:.4f}")


--- ARIMAX Model: NFT Volume (Y) explained by Gas Price (X) [Full Adjustment] ---


C:\Users\Devi\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\Devi\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


                                SARIMAX Results                                 
Dep. Variable:     nft_sales_volume_eth   No. Observations:                  180
Model:                 SARIMAX(1, 1, 1)   Log Likelihood               -1627.866
Date:                  Sun, 02 Nov 2025   AIC                           3269.731
Time:                          22:37:04   BIC                           3291.964
Sample:                      05-06-2025   HQIC                          3278.748
                           - 11-01-2025                                         
Covariance Type:                    opg                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
avg_gas_price_gwei    78.6995    159.048      0.495      0.621    -233.029     390.428
eth_price_usd          0.2268      0.667      0.340      0.734      -1.080       1.534
fear

In [ ]:
# # --- Chunk 3: ARIMAX Model Fitting (SIMPLIFIED ADJUSTMENT) ---

# print("\n--- ARIMAX Model: NFT Volume (Y) explained by Gas Price (X) [SIMPLIFIED ADJUSTMENT] ---")

# # RE-DEFINE CONFOUNDERS FOR ARIMAX: We prioritize the essential controls (ETH Price and Sentiment)
# ARIMAX_CONFOUNDERS = ['eth_price_usd', 'fear_greed_score']
# ARIMAX_EXOG_VARS = PRIMARY_EXOG + ARIMAX_CONFOUNDERS

# endog = df[DEPENDENT_VAR]
# exog = df[ARIMAX_EXOG_VARS] 

# # Fit the SARIMAX model: order=(p, d, q) = (1, 1, 1)
# model = SARIMAX(endog, exog=exog, order=(1,1,1), enforce_stationarity=False, enforce_invertibility=False)
# results = model.fit(disp=False)

# # Check for successful convergence before proceeding
# if results is None:
#     print("\nFATAL ERROR: ARIMAX model failed to converge (returned None). Check data stability.")
#     # Set a placeholder variable to avoid crashing Chunk 4 later
#     ARIMAX_RESULTS = None
# else:
#     print(results.summary())

#     # Key Interpretation: Look at the coefficient for the primary variable (Gas Price)
#     print(f"\nADJUSTED Gas Price Coefficient (Isolated Effect): {results.params['avg_gas_price_gwei']:.4f}")
    
#     # Store the results object for safe use in Chunk 4
#     ARIMAX_RESULTS = results


--- ARIMAX Model: NFT Volume (Y) explained by Gas Price (X) [SIMPLIFIED ADJUSTMENT] ---


C:\Users\Devi\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\Devi\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


                                SARIMAX Results                                 
Dep. Variable:     nft_sales_volume_eth   No. Observations:                  180
Model:                 SARIMAX(1, 1, 1)   Log Likelihood               -1627.387
Date:                  Sun, 02 Nov 2025   AIC                           3266.774
Time:                          19:54:52   BIC                           3285.831
Sample:                      05-06-2025   HQIC                          3274.503
                           - 11-01-2025                                         
Covariance Type:                    opg                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
avg_gas_price_gwei    70.0572    193.297      0.362      0.717    -308.799     448.913
eth_price_usd          0.0850      0.683      0.124      0.901      -1.254       1.424
fear

In [27]:
# --- Chunk 4: Interrupted Time Series Analysis (Iterative OLS) ---
print("\n--- OLS Interrupted Time Series Analysis (Testing Multiple Events) ---")

# DEFINE MULTIPLE INTERVENTION PERIODS based on your data spikes
# Use dates that show significant gas activity in the post_period.
INTERVENTIONS = [
    {
        'name': 'Initial Spike (Aug)', 
        'pre': ['2025-05-05', '2025-07-20'], 
        'post': ['2025-07-21', '2025-08-21']
    },
    {
        'name': 'Mid-Autumn Volatility',
        'pre': ['2025-08-10', '2025-09-10'],
        'post': ['2025-09-11', '2025-10-11']
    },
    {
        'name': 'Late-Year Rush',
        'pre': ['2025-09-20', '2025-10-25'],
        'post': ['2025-10-26', '2025-11-20']
    }
]

# Define OLS Model Variables (Y and Controls C)
y = df[DEPENDENT_VAR]
CONTROL_VARS = ['avg_gas_price_gwei', 'eth_price_usd', 'fear_greed_score', 'new_addresses']
X_vars_base = df[CONTROL_VARS].copy() # Base controls

# --- Run Analysis for Each Intervention ---
for intervention in INTERVENTIONS:
    
    print("\n" + "="*40)
    print(f"ANALYZING EVENT: {intervention['name']}")
    
    # 1. Create the Intervention Dummy (Z) for this specific period
    pre_start, pre_end = pd.to_datetime(intervention['pre'])
    post_start, post_end = pd.to_datetime(intervention['post'])

    # Ensure the combined data covers both periods
    df_event = df.loc[pre_start:post_end].copy()
    
    # Z_Intervention = 1 during the post-period, 0 otherwise
    df_event['Z_Intervention'] = np.where(df_event.index >= post_start, 1, 0)
    
    # 2. Define the Model Inputs
    y_event = df_event[DEPENDENT_VAR]
    X_event = df_event[['Z_Intervention'] + CONTROL_VARS]
    X_event = sm.add_constant(X_event)

    # 3. Fit the OLS Model
    try:
        ols_model = sm.OLS(y_event, X_event).fit()
        
        # 4. Print Key Results
        intervention_coeff = ols_model.params['Z_Intervention']
        p_value_ols = ols_model.pvalues['Z_Intervention']

        print(f"Level Shift Coef (Z_Intervention): {intervention_coeff:.2f} ETH")
        print(f"P-value for Shift: {p_value_ols:.4f}")

        if p_value_ols < 0.05:
            print(">>> CONCLUSION: Significant Level Shift (Effect Confirmed).")
        else:
            print(">>> CONCLUSION: No Significant Level Shift (Effect Absent).")

    except Exception as e:
        print(f"ERROR: Model failed to fit for {intervention['name']}: {e}")


--- OLS Interrupted Time Series Analysis (Testing Multiple Events) ---

ANALYZING EVENT: Initial Spike (Aug)
Level Shift Coef (Z_Intervention): 3865.67 ETH
P-value for Shift: 0.0315
>>> CONCLUSION: Significant Level Shift (Effect Confirmed).

ANALYZING EVENT: Mid-Autumn Volatility
Level Shift Coef (Z_Intervention): 174.98 ETH
P-value for Shift: 0.6336
>>> CONCLUSION: No Significant Level Shift (Effect Absent).

ANALYZING EVENT: Late-Year Rush
Level Shift Coef (Z_Intervention): -1429.85 ETH
P-value for Shift: 0.0341
>>> CONCLUSION: Significant Level Shift (Effect Confirmed).
